Mount to MyDrive:

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Download repo and install dependencies:

In [ ]:
import os

%cd /content/gdrive/MyDrive

if os.getcwd() == '/content/gdrive/MyDrive/NLP-Business-News-Summarization':
  %cd ..
  !rm -r ./NLP-Business-News-Summarization/

# Remove Colab default sample_data
if os.path.exists('/content/sample_data'):
  !rm -r /content/sample_data

repo_name = "NLP-Business-News-Summarization" # Repository Name

if not os.path.isdir('./' + repo_name):
  # Clone GitHub Files
  git_path = 'https://github.com/christianwhollar/NLP-Business-News-Summarization.git'
  !git clone "{git_path}"

# Change to new directory
if not os.path.basename(os.getcwd()) == repo_name:
  %cd "{repo_name}"

  # Install dependencies from requirements.txt file
  !pip install -r requirements.txt
  !pip install transformers[torch]
  !python -c "from accelerate.utils import write_basic_config; write_basic_config(mixed_precision='fp16')"

Upload Kaggle Creds:

In [ ]:
from google.colab import files

if 'kaggle.json' not in os.listdir():
  files.upload()
  ! mkdir ~/.kaggle
  ! cp kaggle.json /content/gdrive/MyDrive/NLP-Business-News-Summarization/data/
  ! chmod 600 /content/gdrive/MyDrive/NLP-Business-News-Summarization/data/kaggle.json
  ! cp kaggle.json ~/.kaggle/
  ! chmod 600 ~/.kaggle/kaggle.json

Running setup.py will do the following:
* clean the data directory
* download the datasets from Kaggle
* process the data and build HuggingFace datasets
* train the generative model (train/val datasets)
* test the generative model (test dataset)
* test the extractive model

In [ ]:
%run scripts/setup.py

Launch the streamlit app:

In [ ]:
!streamlit run app/SummarizeBusinessArticle.py